In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eye-images-and-hb-level/Eye_Features.xlsx
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1709720371526.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1709800783595.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1709980654965.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1709709970921.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1709970267160.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1709985657508.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1710049249967.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1709880362990.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1709789649074.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1710050083980.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1710067886551.jpg
/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked/1709880494391.jpg
/kaggle/

In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [3]:
# Read Excel file and select needed columns
df = pd.read_excel('/kaggle/input/eye-images-and-hb-level/Eye_Features.xlsx')
df = df[['Image_Name', 'Hb Value']]

In [4]:
# Split data into train (approx. 70%), validation (approx. 15%), and test (15%)
train_df, test_df = train_test_split(df, test_size=0.15, random_state=42)
train_df, val_df  = train_test_split(train_df, test_size=0.176, random_state=42)

In [5]:
# Function to load and preprocess images
def load_image_and_label(image_file, label):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])  # Resize for ResNet50
    image = image / 255.0  # Normalize to [0, 1]
    return image, label

# Function to create TensorFlow datasets
def create_tf_dataset(df, images_folder, batch_size=16, shuffle=True):
    image_paths = [os.path.join(images_folder, img) for img in df['Image_Name'].values]
    labels = df['Hb Value'].values.astype(np.float32)
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(load_image_and_label, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(df))
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Define your image folder path and create datasets
images_folder = "/kaggle/input/eye-images-and-hb-level/Eye Masked/Eye Masked"
train_ds = create_tf_dataset(train_df, images_folder, batch_size=16, shuffle=True)
val_ds   = create_tf_dataset(val_df,   images_folder, batch_size=16, shuffle=False)
test_ds  = create_tf_dataset(test_df,  images_folder, batch_size=16, shuffle=False)

In [7]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load ResNet50 base (without top) with pretrained ImageNet weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers initially
for layer in base_model.layers:
    layer.trainable = False

# Build the regression head on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', name='feature_dense')(x)  # Named layer for later feature extraction
x = Dropout(0.5)(x)
predictions = Dense(1, activation='linear')(x)  # Linear activation for regression

# Construct the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='mean_absolute_error', metrics=['mae'])

# Train the new head (freeze base model)
history = model.fit(train_ds, validation_data=val_ds, epochs=50)


Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 16s 397ms/step - loss: 9.8275 - mae: 9.8275 - val_loss: 4.5349 - val_mae: 4.5349
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 3.5017 - mae: 3.5017 - val_loss: 1.1874 - val_mae: 1.1874
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 1.7976 - mae: 1.7976 - val_loss: 1.1205 - val_mae: 1.1205
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 1.7959 - mae: 1.7959 - val_loss: 1.2366 - val_mae: 1.2366
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 1.7618 - mae: 1.7618 - val_loss: 1.1157 - val_mae: 1.1157
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 1.6798 - mae: 1.6798 - val_loss: 1.1630 - val_mae: 1.1630
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 1.8137 - mae: 1.8137 - val_loss: 1.1705 - val_mae: 1.1705
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 1.6620 - mae: 1.6620 - val_loss: 1.1177 - val_mae: 1.1177
Epoch 9/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 1.7

In [8]:
# Unfreeze the last 20 layers of the base model for fine tuning
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Re-compile the model with a lower learning rate for fine tuning
model.compile(optimizer=Adam(learning_rate=1e-5), loss='mean_absolute_error', metrics=['mae'])

# Fine tune the model further
history_ft = model.fit(train_ds, validation_data=val_ds, epochs=100)

# Optionally, evaluate the fine tuned model on the test dataset
test_mae = model.evaluate(test_ds)
print("Test MAE after fine tuning:", test_mae)


Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 25s 614ms/step - loss: 3.1394 - mae: 3.1394 - val_loss: 1.5544 - val_mae: 1.5544
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 1.6322 - mae: 1.6322 - val_loss: 1.6966 - val_mae: 1.6966
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 1.7059 - mae: 1.7059 - val_loss: 1.5015 - val_mae: 1.5015
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 1.6917 - mae: 1.6917 - val_loss: 1.1230 - val_mae: 1.1230
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 1.7123 - mae: 1.7123 - val_loss: 1.3708 - val_mae: 1.3708
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 1.6786 - mae: 1.6786 - val_loss: 1.8468 - val_mae: 1.8468
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 1.5774 - mae: 1.5774 - val_loss: 2.4016 - val_mae: 2.4016
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 1.7138 - mae: 1.7138 - val_loss: 2.8609 - val_mae: 2.8609
Epoch 9/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - 

In [9]:
# Create a feature extractor model that outputs the activations of the "feature_dense" layer
feature_extractor = Model(inputs=model.input, outputs=model.get_layer("feature_dense").output)

# Define a helper function to extract features from a TF dataset
def extract_features(dataset, extractor_model):
    features_list = []
    labels_list = []
    for batch_images, batch_labels in dataset:
        features = extractor_model.predict(batch_images)
        features_list.append(features)
        labels_list.append(batch_labels.numpy())
    features_array = np.concatenate(features_list, axis=0)
    labels_array = np.concatenate(labels_list, axis=0)
    return features_array, labels_array

# Extract features and labels from each dataset
train_features, train_labels = extract_features(train_ds, feature_extractor)
val_features, val_labels     = extract_features(val_ds, feature_extractor)
test_features, test_labels   = extract_features(test_ds, feature_extractor)

print("Train features shape:", train_features.shape)
print("Validation features shape:", val_features.shape)
print("Test features shape:", test_features.shape)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Train features sha

In [10]:
!pip install pymrmr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pymrmr: filename=pymrmr-0.1.11-cp310-cp310-linux_x86_64.whl size=390770 sha256=6d3184752a6592250500683255c6d405d7c59a7d39be2be9add1eedaf7dd71dc
  Stored in directory: /root/.cache/pip/wheels/46/ae/55/4a2479c5f0de7eb363fe970cb18e4a750e03e4e63b1b5c2005
Successfully built pymrmr


In [11]:
import pandas as pd
import numpy as np
import pymrmr

# Assume 'train_features' and 'train_labels' are from the feature extractor
# Create a DataFrame with columns f0, f1, ..., and the target "Hb Value"
train_feat_df = pd.DataFrame(train_features, columns=[f"f{i}" for i in range(train_features.shape[1])])
train_feat_df['Hb Value'] = train_labels

# Discretize each feature column into 10 quantile bins.
for col in train_feat_df.columns[:-1]:  # Exclude the target column
    # qcut will bin the continuous values into 10 categories.
    train_feat_df[col] = pd.qcut(train_feat_df[col], q=10, duplicates='drop').cat.codes

# Discretize the target as well.
train_feat_df['Hb Value'] = pd.qcut(train_feat_df['Hb Value'], q=10, duplicates='drop').cat.codes


In [14]:
# Define the number of top features you wish to select
num_top_features = 100

# Run mRMR to select features
selected_features = pymrmr.mRMR(train_feat_df, 'MIQ', num_top_features)
print("Selected features:", selected_features)

# Convert feature names (e.g., "f23") to column indices
selected_indices = []

for feat in selected_features:
    # Ensure the feature name starts with 'f' and is followed by digits
    if feat.startswith('f') and feat[1:].isdigit():
        selected_indices.append(int(feat[1:]))
    else:
        print(f"Skipping non-standard feature: {feat}")

print("Selected feature indices:", selected_indices)


Selected features: ['f161', 'f855', 'f281', 'f455', 'f175', 'f366', 'f634', 'f596', 'f914', 'f806', 'f61', 'f608', 'f302', 'f721', 'f375', 'f343', 'f702', 'Hb Value', 'f555', 'f558', 'f758', 'f613', 'f393', 'f354', 'f546', 'f401', 'f981', 'f222', 'f706', 'f90', 'f965', 'f747', 'f980', 'f915', 'f480', 'f1011', 'f120', 'f847', 'f119', 'f53', 'f597', 'f14', 'f896', 'f435', 'f665', 'f310', 'f416', 'f925', 'f185', 'f215', 'f854', 'f493', 'f824', 'f678', 'f223', 'f789', 'f203', 'f396', 'f158', 'f550', 'f728', 'f389', 'f649', 'f44', 'f100', 'f136', 'f884', 'f904', 'f278', 'f669', 'f974', 'f379', 'f619', 'f694', 'f76', 'f621', 'f373', 'f1023', 'f881', 'f600', 'f200', 'f359', 'f81', 'f168', 'f361', 'f228', 'f966', 'f748', 'f580', 'f31', 'f523', 'f365', 'f709', 'f37', 'f110', 'f672', 'f193', 'f757', 'f442', 'f1']
Skipping non-standard feature: Hb Value
Selected feature indices: [161, 855, 281, 455, 175, 366, 634, 596, 914, 806, 61, 608, 302, 721, 375, 343, 702, 555, 558, 758, 613, 393, 354, 546,

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Slice the continuous features using the selected indices
X_train = train_features[:, selected_indices]
X_val   = val_features[:, selected_indices]
X_test  = test_features[:, selected_indices]

# Train the regression model on the training set
regressor = RandomForestRegressor(random_state=42)
regressor.fit(X_train, train_labels)

# Predict on the validation set
val_preds = regressor.predict(X_val)
val_mae = mean_absolute_error(val_labels, val_preds)
print("Validation MAE:", val_mae)

# Predict on the test set
test_preds = regressor.predict(X_test)
test_mae = mean_absolute_error(test_labels, test_preds)
print("Test MAE:", test_mae)


Validation MAE: 1.6442787539372674
Test MAE: 2.1348196991154405


In [16]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

# Slice the continuous features using the selected indices from mRMR
X_train = train_features[:, selected_indices]
X_val   = val_features[:, selected_indices]
X_test  = test_features[:, selected_indices]

# Initialize XGBoost regressor with typical hyperparameters
xgb_regressor = xgb.XGBRegressor(
    objective='reg:squarederror',  # For regression tasks
    n_estimators=200,               # Number of trees
    learning_rate=0.05,             # Step size shrinkage
    max_depth=6,                    # Maximum depth of a tree
    random_state=42
)

# Train the XGBoost model, using the validation set for early stopping
xgb_regressor.fit(
    X_train, 
    train_labels, 
    eval_set=[(X_val, val_labels)],
    early_stopping_rounds=10,
    verbose=True
)

# Make predictions on the validation and test sets
val_preds = xgb_regressor.predict(X_val)
test_preds = xgb_regressor.predict(X_test)

# Calculate Mean Absolute Error (MAE) for validation and test sets
val_mae = mean_absolute_error(val_labels, val_preds)
test_mae = mean_absolute_error(test_labels, test_preds)

print("Validation MAE:", val_mae)
print("Test MAE:", test_mae)


[0]	validation_0-rmse:1.40130
[1]	validation_0-rmse:1.42073
[2]	validation_0-rmse:1.42909
[3]	validation_0-rmse:1.44406
[4]	validation_0-rmse:1.45957
[5]	validation_0-rmse:1.47045
[6]	validation_0-rmse:1.48830
[7]	validation_0-rmse:1.49834
[8]	validation_0-rmse:1.51242
[9]	validation_0-rmse:1.52644
Validation MAE: 1.1384141
Test MAE: 1.5398421


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [18]:
!pip install ace_tools

In [19]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error

# List of regressors to try
regressors = {
    "RandomForest": RandomForestRegressor(random_state=42),
    "GradientBoosting": GradientBoostingRegressor(random_state=42),
    "AdaBoost": AdaBoostRegressor(random_state=42),
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "SVR": SVR(),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "KNN": KNeighborsRegressor(),
    "XGBoost": XGBRegressor(random_state=42, verbosity=0),
    "LightGBM": LGBMRegressor(random_state=42),
    "CatBoost": CatBoostRegressor(verbose=0, random_state=42)
}

# Slice the continuous features using the selected indices
X_train = train_features[:, selected_indices]
X_val   = val_features[:, selected_indices]
X_test  = test_features[:, selected_indices]

# Dictionary to store MAE results
results = {}

# Iterate through each model
for name, model in regressors.items():
    print(f"Training {name}...")
    model.fit(X_train, train_labels)
    
    # Validation Predictions
    val_preds = model.predict(X_val)
    val_mae = mean_absolute_error(val_labels, val_preds)
    
    # Test Predictions
    test_preds = model.predict(X_test)
    test_mae = mean_absolute_error(test_labels, test_preds)
    
    results[name] = {"Validation MAE": val_mae, "Test MAE": test_mae}
    print(f"{name} - Validation MAE: {val_mae:.4f} | Test MAE: {test_mae:.4f}")

# Display results
import pandas as pd
results_df = pd.DataFrame(results).T
import ace_tools as tools; tools.display_dataframe_to_user(name="Regressor Model Performance", dataframe=results_df)


Training RandomForest...
RandomForest - Validation MAE: 1.6443 | Test MAE: 2.1348
Training GradientBoosting...
GradientBoosting - Validation MAE: 1.7060 | Test MAE: 2.1260
Training AdaBoost...
AdaBoost - Validation MAE: 1.7615 | Test MAE: 2.0239
Training LinearRegression...
LinearRegression - Validation MAE: 1.7330 | Test MAE: 2.3213
Training Ridge...
Ridge - Validation MAE: 1.5815 | Test MAE: 2.0949
Training Lasso...
Lasso - Validation MAE: 1.1232 | Test MAE: 1.5344
Training SVR...
SVR - Validation MAE: 1.4908 | Test MAE: 2.0039
Training DecisionTree...
DecisionTree - Validation MAE: 1.9049 | Test MAE: 2.3885
Training KNN...
KNN - Validation MAE: 1.6682 | Test MAE: 2.1761
Training XGBoost...
XGBoost - Validation MAE: 1.7348 | Test MAE: 2.0379
Training LightGBM...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000350 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9024
[LightGBM] [Info] Number of d

ModuleNotFoundError: No module named 'ace_tools'